In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *
from spytial.annotations import flag

There are many ways to represent graphs, including explicitly encoding the edges in the graph node, adjacency matrices, adjacency lists, edge lists.

## Unweighted Graphs from Adjacency Matrix

![unweighted-dir-graph](./img/directed-graph.png)


** View from Adjacency List (**c**) to directed graph (**a**)

In [2]:
adj_matrix = [[0, 1, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 1, 1],
              [0, 1, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1]]


# These are 1 indexed instead of 0 indexed.
directedgraph = inferredEdge(selector="{i, j : int | (some xs : list | (some ys : list |  (xs->i->ys) in idx and (ys -> j -> ({x : int | @num:x = 1})  in idx  )))}", name="")(adj_matrix)
directedgraph = hideAtom(selector="list-int")(directedgraph)
diagram(directedgraph)

Alternately, view from adjacency matrix (**c**) to adjacency list (**b**).

The adjacecy list representation in (**b**) violates one core principle of `spytial`: The same element shows up more than once in the diagram.

There are two options here:
- Write a relationalizer for adjacency matrices to duplicate if needed.
- Show the representation in some other way (without duplication): **HOWEVER, THIS IS EFFECTIVELY THE GRAPH IN (A)**.



In [3]:
@attribute(field='key')
class GNode:
    def __init__(self, key):
        self.key = key



@inferredEdge(selector='{u : GNode, w : int, v : GNode | (some t : tuple | t.t0 = u and t.t1 = v and t.t2 = w) }', name='edge')
@hideAtom(selector='list + tuple + Graph + int + str')
class Graph:
    def __init__(self):
        self.adj = []  # adjacency list representation

    def addEdge(self, u: GNode, v: GNode, w: int):
        self.adj.append((u, v, w))


adj_list = [[j for j,val in enumerate(row) if val] for row in adj_matrix]  # 0-indexed

g = Graph()                       # create one graph once
nodes = {i: GNode(i) for i in range(len(adj_list))}  # reuse node objects
node_list = list(nodes.values())

print(nodes)

for i, nbrs in enumerate(adj_list):
    for j in nbrs:
        g.addEdge(nodes[i], nodes[j], 1)

diagram(g)


{0: <__main__.GNode object at 0x106d7d3d0>, 1: <__main__.GNode object at 0x106d7e630>, 2: <__main__.GNode object at 0x10793da00>, 3: <__main__.GNode object at 0x10793ec90>, 4: <__main__.GNode object at 0x10793f8f0>, 5: <__main__.GNode object at 0x10793fd40>}


# Minimum Spanning Tree



In [4]:
@inferredEdge(selector='{u : GNode, w : int, v : GNode | (some t : tuple & (int.((MSTGraph.mst_edges).idx)) | t.t0 = u and t.t1 = v and t.t2 = w) }', name='mst', color='green')
class MSTGraph(Graph):
    def __init__(self):
        super().__init__()  # Initialize the parent Graph class
        self.mst_edges = []  # Store edges that are part of the MST

    def compute_mst(self):
        """Compute the MST using Kruskal's algorithm and store the edges."""
        parent = {}
        rank = {}

        # Helper functions for union-find
        def find(node):
            if parent[node] != node:
                parent[node] = find(parent[node])  # Path compression
            return parent[node]

        def union(node1, node2):
            root1 = find(node1)
            root2 = find(node2)
            if root1 != root2:
                if rank[root1] > rank[root2]:
                    parent[root2] = root1
                elif rank[root1] < rank[root2]:
                    parent[root1] = root2
                else:
                    parent[root2] = root1
                    rank[root1] += 1

        # Initialize union-find structures for ALL nodes that appear in edges
        all_nodes = set()
        for u, v, w in self.adj:
            all_nodes.add(u)
            all_nodes.add(v)
        
        for node in all_nodes:
            parent[node] = node
            rank[node] = 0

        # Sort edges by weight
        edges = sorted(self.adj, key=lambda x: x[2])  # (u, v, weight)

        # Kruskal's algorithm
        for u, v, w in edges:
            if find(u) != find(v):
                union(u, v)
                self.mst_edges.append((u, v, w))  # Add edge to MST



![mst](img/minimum-spanning-tree.png)

In [ ]:


# Create the graph
mst_graph = MSTGraph()

a = GNode("a")
b = GNode("b")
c = GNode("c")
d = GNode("d")
e = GNode("e")
f = GNode("f")
g = GNode("g")
h = GNode("h")
i = GNode("i")

node_list = [a, b, c, d, e, f, g, h, i]
nodes = {node.key: node for node in node_list}


mst_graph.addEdge(a, b, 4)
mst_graph.addEdge(a, h, 8)
mst_graph.addEdge(c,d,7)
mst_graph.addEdge(c,f,4)
mst_graph.addEdge(c,i,2)
mst_graph.addEdge(b,c,8)
mst_graph.addEdge(b,h,11)
mst_graph.addEdge(d,e,9)
mst_graph.addEdge(d,f,14)
mst_graph.addEdge(e,f,10)
mst_graph.addEdge(f,g,2)
mst_graph.addEdge(g,h,1)
mst_graph.addEdge(g,i,6)
mst_graph.addEdge(h,i,7)

# Compute the MST
mst_graph.compute_mst()


diagram(mst_graph)

{'a': <__main__.GNode object at 0x106c4dbe0>, 'b': <__main__.GNode object at 0x10793deb0>, 'c': <__main__.GNode object at 0x10793db80>, 'd': <__main__.GNode object at 0x10793dbe0>, 'e': <__main__.GNode object at 0x10793d3a0>, 'f': <__main__.GNode object at 0x10793d2b0>, 'g': <__main__.GNode object at 0x10793d370>, 'h': <__main__.GNode object at 0x106c4c770>, 'i': <__main__.GNode object at 0x10793d8b0>}
